In [2]:
# necessary for DenodoConnection
#%pip install psycopg2-binary
#%pip install shap
#%pip install scikit-optimize

# generate train and validation sets
# during training, make K models via k-folds cross validation
# during evaluation, evaluate on validation set (completely left out).

# stages 1,3, 4 for Corn Brazil Summer, advancements made on hybrids
# do we combine stages to make 1 model?

In [3]:
# import packages
import os
import sys

import pandas as pd, numpy as np
import argparse

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.performance_lib import predictive_advancement_lib
from libs.performance_lib import performance_helper

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
DKU_DST_ap_data_sector = 'CORN_BRAZIL_SUMMER'
DKU_DST_analysis_type = 'SingleExp'
input_years = ['2018','2019','2020','2021','2022']
bucket = 'us.com.syngenta.ap.nonprod'

# convert each entry in input years to an integer since they come in as str.
if isinstance(input_years, list):
    input_years_as_int = [int(val) for val in input_years]
else:
    input_years_as_int = [int(input_years)]

In [5]:
# when testing, we read directly from S3, bucket is provided as a separate input to function, don't include in input args. 
input_args = [
    '--s3_input_pred_adv_data_folder', 'uat/dme/performance/compute_pred_adv_data_collected/data/'
]

parser = argparse.ArgumentParser(description='app inputs and outputs')
parser.add_argument('--s3_input_pred_adv_data_folder', type=str,
                    help='s3 input pred_adv_data folder', required=True)
args = parser.parse_args(input_args)

In [7]:
potential_fnames = predictive_advancement_lib.load_potential_fnames(prefix='new_')

df_input_piv = predictive_advancement_lib.load_and_preprocess_all_inputs_ml(
    args,
    DKU_DST_ap_data_sector,
    DKU_DST_analysis_type,
    potential_fnames,
    years_to_load=input_years,
    read_from_s3=1,
    bucket=bucket
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


(17273, 41)
(82029, 9)
(25371, 45)
(145042, 9)
(23423, 47)
(118128, 9)
(9077, 54)
(8306, 47)
(11, 4)
(77136, 9)
(9926, 53)
(9217, 38)
(35, 4)
(61583, 9)


In [8]:
df_input_piv.shape

(85070, 103)

In [9]:
df_input_piv[df_input_piv['material_type_simple'] == 'entry'][['entry_identifier','current_stage','analysis_year']].groupby(by=['current_stage','analysis_year']).count()

entry_identifier
current_stage analysis_year                  
1.0           2018                      12790
              2019                      21963
              2020                      19765
              2021                       5603
              2022                       4177
2.0           2018                        197
3.0           2018                       2865
              2019                       2264
              2020                       2395
              2021                       1259
              2022                       2898
4.0           2018                       1303
              2019                        761
              2020                        717
              2021                       1821
              2022                       2465
5.0           2018                         76
              2019                        261
              2020                        216
              2021                        225
              2022                        137
6.0           2018                          6
              2019                         23
              2020                         86
              2021                         29
              2022                         51
7.0           2018                         32
              2019                         77
              2020                         75
              2021                         99
              2022                         53

In [10]:
for col in df_input_piv.columns:
    print(col, df_input_piv[col].count())

ap_data_sector 85070
analysis_year 85070
entry_identifier 85070
decision_group_rm 85070
material_type_simple 85070
chkfl_ERHTN 16770
chkfl_GMSTP 83834
chkfl_GWTPN 83706
chkfl_HAVPN 19692
chkfl_LRTLR 69022
chkfl_PLHTN 16811
chkfl_STD_N 21339
chkfl_STD_P 19611
chkfl_STKLR 69274
chkfl_YGHMN 83753
chkfl_YGMRN 83753
chkfl_YGSMN 83753
result_ERHTN 16770
result_GMSTP 83834
result_GWTPN 83706
result_HAVPN 19692
result_LRTLR 69022
result_PLHTN 16811
result_STD_N 21339
result_STD_P 19611
result_STKLR 69274
result_YGHMN 83753
result_YGMRN 83753
result_YGSMN 83753
result_diff_ERHTN 16770
result_diff_GMSTP 83834
result_diff_GWTPN 83706
result_diff_HAVPN 19692
result_diff_LRTLR 69022
result_diff_PLHTN 16811
result_diff_STD_N 21339
result_diff_STD_P 19611
result_diff_STKLR 69274
result_diff_YGHMN 83753
result_diff_YGMRN 83753
result_diff_YGSMN 83753
current_stage 84689
next_stage 84689
prev_stage 6738
third_stage 84689
was_adv 84689
was_adv_next 84689
chkfl_TWSMN 45464
cpifl 12470
result_TWSMN 45464


In [11]:
df_input_piv[['was_adv','current_stage','analysis_year']].groupby(by=['current_stage','analysis_year']).sum()

was_adv
current_stage analysis_year         
1.0           2018             283.0
              2019             158.0
              2020             150.0
              2021             344.0
              2022             185.0
2.0           2018               4.0
3.0           2018              52.0
              2019             144.0
              2020             301.0
              2021              69.0
              2022              44.0
4.0           2018              12.0
              2019              46.0
              2020              82.0
              2021             135.0
              2022              85.0
5.0           2018               2.0
              2019               5.0
              2020               7.0
              2021              16.0
              2022               0.0
6.0           2018               0.0
              2019               0.0
              2020               0.0
              2021               0.0
              2022               0.0
7.0           2018               0.0
              2019               0.0
              2020               0.0
              2021               0.0
              2022               0.0

In [15]:
#### FILTER TO DESIRED DATA for training
# for soy brazil, focus only on stage 3 and 4.
df_piv = df_input_piv[(df_input_piv['material_type_simple'] == 'entry') &
                      ((df_input_piv['current_stage']==1) | (df_input_piv['current_stage']==3) | (df_input_piv['current_stage'] == 4))]


if 'decision_group' not in df_piv.columns:
    df_piv['decision_group'] = 'na'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df_piv.head()

,ap_data_sector,analysis_year,entry_identifier,decision_group_rm,material_type_simple,chkfl_ERHTN,chkfl_GMSTP,chkfl_GWTPN,chkfl_HAVPN,chkfl_LRTLR,...,STD_N_prediction,STD_N_stderr,STKLR_count,STKLR_prediction,STKLR_stderr,YGSMN_count,YGSMN_prediction,YGSMN_stderr,rm_estimate,decision_group
1,CORN_BRAZIL_SUMMER,2018,EMF0900174727,-10.0,entry,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na
2,CORN_BRAZIL_SUMMER,2018,EMF0900728763,-10.0,entry,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na
3,CORN_BRAZIL_SUMMER,2018,EMF0900873340,-10.0,entry,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na
4,CORN_BRAZIL_SUMMER,2018,EMF0901511996,-10.0,entry,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na
5,CORN_BRAZIL_SUMMER,2018,EMF1100005676,-10.0,entry,0.0,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na


In [21]:
# get columns used for training/evaluation + meta columns
stratify_cols = ['analysis_year','current_stage']

# split into training and validation sets. Use stratified sampling.
df_tr_all, df_val_all = predictive_advancement_lib.stratified_train_test_split(df_piv, stratify_cols)

# provide label for k-fold training in training set. Don't need to split data, just label it once. Do this in a stratified manner as well 
df_tr_all = predictive_advancement_lib.stratified_kfolds(df_tr_all, stratify_cols, n_folds=5)

In [23]:
# save training and validation sets. <- this will change in the pipeline to some other saving methods...
s3_path=os.path.join('uat/dme/performance/reformatting_performance_pipeline_temp_out/data',DKU_DST_ap_data_sector)
local_fpath = '/root/dme_sagemaker/dme_sagemaker/performance_pipeline/preprocess_train_recipes/'
performance_helper.write_to_s3(
    obj=df_tr_all,
    fname='adv_model_training_data.csv', 
    local_fpath=local_fpath,
    s3_path=s3_path
)

performance_helper.write_to_s3(
    df_val_all,
    fname='adv_model_validation_data.csv', 
    local_fpath=local_fpath,
    s3_path=s3_path
)
### end preprocessing_training.py

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
